# 随机指标(KDJ)
    
    它综合了动量观念、强弱指标及移动平均线的优点，用来度量股价脱离价格正常范围的变异程度。
    
    KDJ指标考虑的不仅是收盘价，而且有近期的最高价和最低价，这避免了仅考虑收盘价而忽视真正波动幅度的弱点

### 原理
    
    随机指标(KDJ)一般是根据统计学的原理，通过一个特定的周期（常为9日、9周等）内出现过的最高价、最低价及最后一个计算周期的收盘价及这三者之间的比例关系，来计算最后一个计算周期的未成熟随机值RSV，然后根据平滑移动平均线的方法来计算K值、D值与J值，并绘成曲线图来研判股票走势。
    
    随机指标(KDJ)是以最高价、最低价及收盘价为基本数据进行计算，得出的K值、D值和J值分别在指标的坐标上形成的一个点，连接无数个这样的点位，就形成一个完整的、能反映价格波动趋势的KDJ指标。它主要是利用价格波动的真实波幅来反映价格走势的强弱和超买超卖现象，在价格尚未上升或下降之前发出买卖信号的一种技术工具。它在设计过程中主要是研究最高价、最低价和收盘价之间的关系，同时也融合了动量观念、强弱指标和移动平均线的一些优点，因此，能够比较迅速、快捷、直观地研判行情。
    
    随机指标(KDJ)最早是以KD指标的形式出现，而KD指标是在威廉指标的基础上发展起来的。不过威廉指标只判断股票的超买超卖的现象，在KDJ指标中则融合了移动平均线速度上的观念，形成比较准确的买卖信号依据。在实践中，K线与D线配合J线组成KDJ指标来使用。由于KDJ线本质上是一个随机波动的观念，故其对于掌握中短期行情走势比较准确。
    
### 计算公式
    
    KDJ指标的计算比较复杂，首先要计算周期(n日、n周等)的未成熟随机值RSV，然后再计算K值、D值和J值。
    
### 未成熟随机值RSV(Raw Stochastic Value)
    
    未成熟随机值是股票的一个概念，RSV指标主要用来分析市场是处于“超买”还是“超卖”状态：RSV高于80%时候市场即为超买状况，行情即将见顶，应当考虑出仓；RSV低于20%时候，市场为超卖状况，行情即将见底，此时可以考虑近仓。

公式: $$RSV(n) = \frac{C_t - L_n}{H_n - L_n} \times 100$$

$C_t$为第n日收盘价, $L_n$为n日内的最低价, $H_n$为n日内的最高价, RSV值始终在l到100间波动

其次，计算K值与D值：

$$K_{当日} = \frac{2}{3}K_{前一日} + \frac{1}{3}RSV_{当日}$$
$$D_{当日} = \frac{2}{3}D_{前一日} + \frac{1}{3}K_{当日}$$

J值依软件设定可能为K-D、3K-2D或3D-2K，此处为3K-2D
J的实质是反映K值和D值的乖离程度，从而领先KD值显示头部或底部。J值范围可超过100和低于0

K线是快速确认线, D线是慢速主干线, J线为方向敏感线

KDJ指标在各类软件中的颜色：K线为白色，D线为黄色，J线为紫色

### 金叉死叉

    金叉主要指股票行情指标的短期线向上穿越长期线的交叉，称之为金叉(K<D)，反之，行情指标的短期线向下穿越长期线的交叉，称之为死叉(K>D)。

In [1]:
import os
import pandas as pd

In [2]:
FILE_DIR = 'Data/stock data/'
stock_file_name_list = []

# 遍历股票文件
for root, dirs, files in os.walk(FILE_DIR):
    if files:
        for file_name in files:
            if '.csv' == file_name[-4:]:
                stock_file_name_list.append(FILE_DIR + file_name)

In [14]:
def get_KDJ(stock, rsv_days, n_day_list):
    """
    获取股票KDJ值, 金叉, 死叉, 以及接下来n天的涨跌幅
    """
    
    low_list = stock['low'].rolling(rsv_days).min()
    
    """
    expanding().min() 或 expanding(1).min()
    
    >>> df = pd.DataFrame({'A': [8, 2, 4, np.nan, 1]})
    >>> df
         A
    0  8.0
    1  2.0
    2  4.0
    3  NaN
    4  1.0
    >>> df['A'].expanding().min()
    0    8.0
    1    2.0
    2    2.0
    3    2.0
    4    1.0
    Name: A, dtype: float64
    
    0    8.0
    1    2.0 == min(2.0, 8.0)
    2    2.0 == min(4.0, 2.0)
    3    2.0 == 2.0
    4    1.0 == min(1.0, 2.0)
    
    >>> df['A'].fillna(value=df['A'].expanding().min())
    0    8.0
    1    2.0
    2    4.0
    3    2.0
    4    1.0
    Name: A, dtype: float64
    """
    # 填补空值为当日过去数据的最小值
    low_list.fillna(value=stock['low'].expanding().min(), inplace=True)
    
    high_list = stock['high'].rolling(rsv_days).max()
    # 填补空值为当日过去数据的最大值
    high_list.fillna(value=stock['high'].expanding().max(), inplace=True)
    
    # 计算未成熟随机指标值
    rsv = (stock['close'] - low_list) / (high_list - low_list) * 100
    
    # alpha = 1/(1+com)
    stock['KDJ_K'] = rsv.ewm(com=2).mean()
    stock['KDJ_D'] = stock['KDJ_K'].ewm(com=2).mean()
    stock['KDJ_J'] = 3 * stock['KDJ_K'] - 2 * stock['KDJ_D']
    
    kdj_position = (stock['KDJ_K'] > stock['KDJ_D'])
    stock['KDJ_金叉死叉'] = ''
    
    # kdj_position.shift(n): 前n日数据
    # kdj_position and not kdj_position.shift()
    # 形成K线与D线的交叉
    stock.loc[kdj_position[kdj_position&-kdj_position.shift()].index, 'KDJ_金叉死叉'] = '金叉'
    stock.loc[kdj_position[-kdj_position&kdj_position.shift()].index, 'KDJ_金叉死叉'] = '死叉'
    for n in n_day_list:
        stock[f'接下来{n}个交易日涨跌幅'] = stock['adjust_price'].shift(-1 * n)/stock['adjust_price'] - 1.0
    stock = stock[stock['KDJ_金叉死叉'] != '']
    return stock

In [15]:
n_day_list=[1, 2, 3, 5, 10, 20, 50]
rsv_days = 9
stock = pd.read_csv(stock_file_name_list[0], parse_dates=[1])
stock.sort_values('date', inplace=True)
stock = get_KDJ(stock, rsv_days, n_day_list)
print(stock[['date', 'KDJ_金叉死叉', 'KDJ_K', 'KDJ_D', 'KDJ_J']+[f'接下来{n}个交易日涨跌幅' for n in n_day_list[:2]]][:10])

          date KDJ_金叉死叉      KDJ_K      KDJ_D      KDJ_J  接下来1个交易日涨跌幅  \
479 2013-01-08       金叉  57.208709  54.308572  63.008984     0.001976   
478 2013-01-09       死叉  54.214322  54.269422  54.104123    -0.009862   
474 2013-01-15       金叉  66.191020  53.757257  91.058545    -0.020329   
473 2013-01-16       金叉  62.724031  56.826007  74.520079    -0.008893   
472 2013-01-17       死叉  55.838021  56.490866  54.532330     0.012961   
470 2013-01-21       金叉  70.132753  61.401800  87.594657     0.007692   
469 2013-01-22       金叉  70.851270  64.567892  83.418025     0.011450   
468 2013-01-23       金叉  75.302232  68.158304  89.590087     0.000943   
467 2013-01-24       金叉  72.417121  69.581159  78.089046     0.004713   
466 2013-01-25       金叉  70.689243  69.951084  72.165563     0.048780   

     接下来2个交易日涨跌幅  
479    -0.007905  
478    -0.031558  
474    -0.029041  
473     0.003953  
472     0.036889  
470     0.019230  
469     0.012404  
468     0.005660  
467     0.053723  
466   

In [19]:
all_stock = pd.DataFrame()
for file_name in stock_file_name_list:
    print(file_name)
    stock = pd.read_csv(file_name, parse_dates=[1])
    stock.sort_values('date', inplace=True)
    stock = get_KDJ(stock, rsv_days, n_day_list)
    # 去除空行
    stock.dropna(inplace=True, how='any')
    all_stock = all_stock.append(stock, ignore_index=True)

Data/stock data/sh600000.csv
Data/stock data/sh600001.csv
Data/stock data/sh600002.csv
Data/stock data/sh600003.csv
Data/stock data/sh600004.csv
Data/stock data/sh600005.csv
Data/stock data/sh600006.csv
Data/stock data/sh600007.csv
Data/stock data/sh600008.csv
Data/stock data/sh600009.csv
Data/stock data/sh600010.csv
Data/stock data/sh600011.csv
Data/stock data/sh600012.csv
Data/stock data/sh600015.csv
Data/stock data/sh600016.csv
Data/stock data/sh600017.csv
Data/stock data/sh600018.csv
Data/stock data/sh600019.csv
Data/stock data/sh600020.csv
Data/stock data/sh600021.csv
Data/stock data/sh600022.csv
Data/stock data/sh600023.csv
Data/stock data/sh600026.csv
Data/stock data/sh600027.csv
Data/stock data/sh600028.csv
Data/stock data/sh600029.csv
Data/stock data/sh600030.csv
Data/stock data/sh600031.csv
Data/stock data/sh600033.csv
Data/stock data/sh600035.csv
Data/stock data/sh600036.csv
Data/stock data/sh600037.csv
Data/stock data/sh600038.csv
Data/stock data/sh600039.csv
Data/stock dat

Data/stock data/sh600319.csv
Data/stock data/sh600320.csv
Data/stock data/sh600321.csv
Data/stock data/sh600322.csv
Data/stock data/sh600323.csv
Data/stock data/sh600325.csv
Data/stock data/sh600326.csv
Data/stock data/sh600327.csv
Data/stock data/sh600328.csv
Data/stock data/sh600329.csv
Data/stock data/sh600330.csv
Data/stock data/sh600331.csv
Data/stock data/sh600332.csv
Data/stock data/sh600333.csv
Data/stock data/sh600335.csv
Data/stock data/sh600336.csv
Data/stock data/sh600337.csv
Data/stock data/sh600338.csv
Data/stock data/sh600339.csv
Data/stock data/sh600340.csv
Data/stock data/sh600343.csv
Data/stock data/sh600345.csv
Data/stock data/sh600346.csv
Data/stock data/sh600348.csv
Data/stock data/sh600350.csv
Data/stock data/sh600351.csv
Data/stock data/sh600352.csv
Data/stock data/sh600353.csv
Data/stock data/sh600354.csv
Data/stock data/sh600355.csv
Data/stock data/sh600356.csv
Data/stock data/sh600357.csv
Data/stock data/sh600358.csv
Data/stock data/sh600359.csv
Data/stock dat

Data/stock data/sh601005.csv
Data/stock data/sh601006.csv
Data/stock data/sh601007.csv
Data/stock data/sh601008.csv
Data/stock data/sh601009.csv
Data/stock data/sh601010.csv
Data/stock data/sh601011.csv
Data/stock data/sh601012.csv
Data/stock data/sh601015.csv
Data/stock data/sh601016.csv
Data/stock data/sh601018.csv
Data/stock data/sh601028.csv
Data/stock data/sh601038.csv
Data/stock data/sh601058.csv
Data/stock data/sh601088.csv
Data/stock data/sh601098.csv
Data/stock data/sh601099.csv
Data/stock data/sh601100.csv
Data/stock data/sh601101.csv
Data/stock data/sh601106.csv
Data/stock data/sh601107.csv
Data/stock data/sh601111.csv
Data/stock data/sh601113.csv
Data/stock data/sh601116.csv
Data/stock data/sh601117.csv
Data/stock data/sh601118.csv
Data/stock data/sh601126.csv
Data/stock data/sh601137.csv
Data/stock data/sh601139.csv
Data/stock data/sh601158.csv
Data/stock data/sh601166.csv
Data/stock data/sh601168.csv
Data/stock data/sh601169.csv
Data/stock data/sh601177.csv
Data/stock dat

Data/stock data/sz000425.csv
Data/stock data/sz000426.csv
Data/stock data/sz000428.csv
Data/stock data/sz000429.csv
Data/stock data/sz000430.csv
Data/stock data/sz000488.csv
Data/stock data/sz000498.csv
Data/stock data/sz000501.csv
Data/stock data/sz000502.csv
Data/stock data/sz000503.csv
Data/stock data/sz000504.csv
Data/stock data/sz000505.csv
Data/stock data/sz000506.csv
Data/stock data/sz000507.csv
Data/stock data/sz000508.csv
Data/stock data/sz000509.csv
Data/stock data/sz000510.csv
Data/stock data/sz000511.csv
Data/stock data/sz000513.csv
Data/stock data/sz000514.csv
Data/stock data/sz000515.csv
Data/stock data/sz000516.csv
Data/stock data/sz000517.csv
Data/stock data/sz000518.csv
Data/stock data/sz000519.csv
Data/stock data/sz000520.csv
Data/stock data/sz000521.csv
Data/stock data/sz000522.csv
Data/stock data/sz000523.csv
Data/stock data/sz000524.csv
Data/stock data/sz000525.csv
Data/stock data/sz000526.csv
Data/stock data/sz000527.csv
Data/stock data/sz000528.csv
Data/stock dat

Data/stock data/sz000832.csv
Data/stock data/sz000833.csv
Data/stock data/sz000835.csv
Data/stock data/sz000836.csv
Data/stock data/sz000837.csv
Data/stock data/sz000838.csv
Data/stock data/sz000839.csv
Data/stock data/sz000848.csv
Data/stock data/sz000850.csv
Data/stock data/sz000851.csv
Data/stock data/sz000852.csv
Data/stock data/sz000856.csv
Data/stock data/sz000858.csv
Data/stock data/sz000859.csv
Data/stock data/sz000860.csv
Data/stock data/sz000861.csv
Data/stock data/sz000862.csv
Data/stock data/sz000863.csv
Data/stock data/sz000866.csv
Data/stock data/sz000868.csv
Data/stock data/sz000869.csv
Data/stock data/sz000875.csv
Data/stock data/sz000876.csv
Data/stock data/sz000877.csv
Data/stock data/sz000878.csv
Data/stock data/sz000880.csv
Data/stock data/sz000881.csv
Data/stock data/sz000882.csv
Data/stock data/sz000883.csv
Data/stock data/sz000885.csv
Data/stock data/sz000886.csv
Data/stock data/sz000887.csv
Data/stock data/sz000888.csv
Data/stock data/sz000889.csv
Data/stock dat

Data/stock data/sz002162.csv
Data/stock data/sz002163.csv
Data/stock data/sz002164.csv
Data/stock data/sz002165.csv
Data/stock data/sz002166.csv
Data/stock data/sz002167.csv
Data/stock data/sz002168.csv
Data/stock data/sz002169.csv
Data/stock data/sz002170.csv
Data/stock data/sz002171.csv
Data/stock data/sz002172.csv
Data/stock data/sz002173.csv
Data/stock data/sz002174.csv
Data/stock data/sz002175.csv
Data/stock data/sz002176.csv
Data/stock data/sz002177.csv
Data/stock data/sz002178.csv
Data/stock data/sz002179.csv
Data/stock data/sz002180.csv
Data/stock data/sz002181.csv
Data/stock data/sz002182.csv
Data/stock data/sz002183.csv
Data/stock data/sz002184.csv
Data/stock data/sz002185.csv
Data/stock data/sz002186.csv
Data/stock data/sz002187.csv
Data/stock data/sz002188.csv
Data/stock data/sz002189.csv
Data/stock data/sz002190.csv
Data/stock data/sz002191.csv
Data/stock data/sz002192.csv
Data/stock data/sz002193.csv
Data/stock data/sz002194.csv
Data/stock data/sz002195.csv
Data/stock dat

Data/stock data/sz002446.csv
Data/stock data/sz002447.csv
Data/stock data/sz002448.csv
Data/stock data/sz002449.csv
Data/stock data/sz002450.csv
Data/stock data/sz002451.csv
Data/stock data/sz002452.csv
Data/stock data/sz002453.csv
Data/stock data/sz002454.csv
Data/stock data/sz002455.csv
Data/stock data/sz002456.csv
Data/stock data/sz002457.csv
Data/stock data/sz002458.csv
Data/stock data/sz002459.csv
Data/stock data/sz002460.csv
Data/stock data/sz002461.csv
Data/stock data/sz002462.csv
Data/stock data/sz002463.csv
Data/stock data/sz002464.csv
Data/stock data/sz002465.csv
Data/stock data/sz002466.csv
Data/stock data/sz002467.csv
Data/stock data/sz002468.csv
Data/stock data/sz002469.csv
Data/stock data/sz002470.csv
Data/stock data/sz002471.csv
Data/stock data/sz002472.csv
Data/stock data/sz002473.csv
Data/stock data/sz002474.csv
Data/stock data/sz002475.csv
Data/stock data/sz002476.csv
Data/stock data/sz002477.csv
Data/stock data/sz002478.csv
Data/stock data/sz002479.csv
Data/stock dat

Data/stock data/sz002733.csv
Data/stock data/sz002735.csv
Data/stock data/sz002736.csv
Data/stock data/sz002737.csv
Data/stock data/sz002738.csv
Data/stock data/sz300001.csv
Data/stock data/sz300002.csv
Data/stock data/sz300003.csv
Data/stock data/sz300004.csv
Data/stock data/sz300005.csv
Data/stock data/sz300006.csv
Data/stock data/sz300007.csv
Data/stock data/sz300008.csv
Data/stock data/sz300009.csv
Data/stock data/sz300010.csv
Data/stock data/sz300011.csv
Data/stock data/sz300012.csv
Data/stock data/sz300013.csv
Data/stock data/sz300014.csv
Data/stock data/sz300015.csv
Data/stock data/sz300016.csv
Data/stock data/sz300017.csv
Data/stock data/sz300018.csv
Data/stock data/sz300019.csv
Data/stock data/sz300020.csv
Data/stock data/sz300021.csv
Data/stock data/sz300022.csv
Data/stock data/sz300023.csv
Data/stock data/sz300024.csv
Data/stock data/sz300025.csv
Data/stock data/sz300026.csv
Data/stock data/sz300027.csv
Data/stock data/sz300028.csv
Data/stock data/sz300029.csv
Data/stock dat

Data/stock data/sz300280.csv
Data/stock data/sz300281.csv
Data/stock data/sz300282.csv
Data/stock data/sz300283.csv
Data/stock data/sz300284.csv
Data/stock data/sz300285.csv
Data/stock data/sz300286.csv
Data/stock data/sz300287.csv
Data/stock data/sz300288.csv
Data/stock data/sz300289.csv
Data/stock data/sz300290.csv
Data/stock data/sz300291.csv
Data/stock data/sz300292.csv
Data/stock data/sz300293.csv
Data/stock data/sz300294.csv
Data/stock data/sz300295.csv
Data/stock data/sz300296.csv
Data/stock data/sz300297.csv
Data/stock data/sz300298.csv
Data/stock data/sz300299.csv
Data/stock data/sz300300.csv
Data/stock data/sz300301.csv
Data/stock data/sz300302.csv
Data/stock data/sz300303.csv
Data/stock data/sz300304.csv
Data/stock data/sz300305.csv
Data/stock data/sz300306.csv
Data/stock data/sz300307.csv
Data/stock data/sz300308.csv
Data/stock data/sz300309.csv
Data/stock data/sz300310.csv
Data/stock data/sz300311.csv
Data/stock data/sz300312.csv
Data/stock data/sz300313.csv
Data/stock dat

In [20]:
print(all_stock[['date', 'KDJ_金叉死叉']+[f'接下来{n}个交易日涨跌幅' for n in n_day_list]][:10])

        date KDJ_金叉死叉  接下来1个交易日涨跌幅  接下来2个交易日涨跌幅  接下来3个交易日涨跌幅  接下来5个交易日涨跌幅  \
0 2013-01-08       金叉     0.001976    -0.007905    -0.029644     0.020751   
1 2013-01-09       死叉    -0.009862    -0.031558     0.020710    -0.001972   
2 2013-01-15       金叉    -0.020329    -0.029041    -0.016457     0.014521   
3 2013-01-16       金叉    -0.008893     0.003953     0.027668     0.047430   
4 2013-01-17       死叉     0.012961     0.036889     0.044865     0.057825   
5 2013-01-21       金叉     0.007692     0.019230     0.020191     0.074999   
6 2013-01-22       金叉     0.011450     0.012404     0.017175     0.078243   
7 2013-01-23       金叉     0.000943     0.005660     0.054717     0.060376   
8 2013-01-24       金叉     0.004713     0.053723     0.065033     0.081997   
9 2013-01-25       金叉     0.048780     0.060037     0.054408     0.121012   

   接下来10个交易日涨跌幅  接下来20个交易日涨跌幅  接下来50个交易日涨跌幅  
0      0.035573      0.172923      0.057309  
1      0.045365      0.183430      0.061141  
2      0.09390

In [21]:
J_stock = all_stock[all_stock['KDJ_金叉死叉'] == '金叉']
S_stock = all_stock[all_stock['KDJ_金叉死叉'] == '死叉']
print('接下来n个交易日 \t 金叉平均涨跌幅 \t 死叉平均涨跌幅 \t 金叉上涨股票比例 \t 死叉上涨股票比例')
for n in n_day_list:
    col = f'接下来{n}个交易日涨跌幅'
    print(
            f'{n} \t\t\t'
            f'{J_stock[col].mean() * 100}% \t '
            f'{S_stock[col].mean() * 100}% \t '
            f'{J_stock[J_stock[col]>0].shape[0]*100.0/J_stock.shape[0]}% \t '
            f'{S_stock[S_stock[col]>0].shape[0]*100.0/S_stock.shape[0]}%'
         )

接下来n个交易日 	 金叉平均涨跌幅 	 死叉平均涨跌幅 	 金叉上涨股票比例 	 死叉上涨股票比例
1 			0.06198705168348344% 	 0.1282283927005927% 	 47.6396481017929% 	 53.26863397844609%
2 			0.13462788861596237% 	 0.462573355002892% 	 48.40319167994112% 	 54.26056536703718%
3 			0.24062382142304434% 	 0.653811495084198% 	 49.09023564785002% 	 54.14658917790442%
5 			0.512987561479601% 	 1.0203259485169565% 	 50.86183105206331% 	 55.36647294476104%
10 			1.46817953803514% 	 1.7602571730776357% 	 54.805434377375484% 	 56.82333690684421%
20 			2.836875658154333% 	 3.09652872721843% 	 56.839936669942915% 	 58.14816904587259%
50 			7.369284533853575% 	 7.787509713793715% 	 60.2042249088541% 	 60.68047170343621%
